# Requirements

In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00


To fine-tune a transformer model, the subsequent steps should be followed:

*   Tokenize data
*   Train the model
*   Evaluate the model







# Step1: Tokenization

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [2]:
def tokenize_function(example):
   tokenizer_output= tokenizer(example['tokens'],truncation=True,is_split_into_words=True)
   return{**tokenizer_output,'labels':example['ner_tags']}


In [3]:
# Load data
dataset=load_dataset("conll2003")

  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

In [5]:
# Prepare the tokenizer
checkpoint="dslim/bert-base-NER"
tokenizer=AutoTokenizer.from_pretrained(checkpoint)

In [6]:
dataset=dataset.map(tokenize_function,batched=True)
dataset= dataset.remove_columns(['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'])

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [7]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3453
    })
})

In [8]:
# Split the data
train_dataset=dataset['train']
val_dataset=dataset['validation']
test_dataset=dataset['test']

# Step2: FineTuning

In [9]:
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments
import evaluate
import numpy as np
from transformers import Trainer
from transformers import DataCollatorWithPadding
import torch

In [10]:
#Dynamic padding
def my_collate(batch):
  input_ids=[example['input_ids'] for example in batch]
  attention_masks=[example['attention_mask'] for example in batch]
  labels=[example['labels'] for example in batch]
  max_length=max(len(x) for x in input_ids)
  #pad all inputs to max length
  padded_input_ids=[lst+[0]*(max_length-len(lst)) for lst in input_ids]
  padded_attention_masks=[lst+[0]*(max_length-len(lst)) for lst in attention_masks]
  padded_labels=[lst+[0]*(max_length-len(lst)) for lst in labels]

  return {'input_ids':torch.tensor(padded_input_ids),'attention_mask':torch.tensor(padded_attention_masks),'labels':torch.tensor(padded_labels)}

In [11]:
#Trainer API
trainig_args=TrainingArguments('bert_trainer')
model=AutoModelForTokenClassification.from_pretrained(checkpoint)

In [12]:
def compute_metrics(eval_preds):
  metric=evaluate.load("accuracy")
  logits,labels=eval_preds
  predictions=np.argmax(logits,axis=-1)
  return metric.compute(predictions=predictions,references=labels)

In [13]:
trainer=Trainer(
    model,
    trainig_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=my_collate,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss


# Test the model

In [ ]:
test_preds=trainer.predict(test_dataset)